In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Import the Libraries

In [3]:
! pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.4 MB/s eta 0:00:00


In [1]:
! pip install langchain==0.1.14 langchain-core==0.1.38 pydantic==1.10.13


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 5.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.2/279.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.8 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.11.3
    Uninstalling pydantic-2.11.3:

In [4]:
import os
import re
import numpy as np
import pandas as pd
from typing import List, Dict, Tuple
import torch
import PyPDF2
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import subprocess
import sys

2025-04-22 10:55:09.317249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745319309.522004      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745319309.573625      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [8]:
import subprocess
import sys

def install_if_needed(package_import_pairs):
    for package, import_name in package_import_pairs:
        try:
            __import__(import_name)
            print(f"{package} already installed.")
        except ImportError:
            print(f"Installing {package}...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Define packages with their actual import names
packages = [
    ("langchain", "langchain"),              # just install latest langchain
    ("faiss-cpu", "faiss"),                  # faiss-cpu for CPU environments like Kaggle
    ("ipywidgets", "ipywidgets"),
    ("pydantic==1.10.13", "pydantic")        # if needed for langchain compatibility
]

install_if_needed(packages)


langchain already installed.
Installing faiss-cpu...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 62.2 MB/s eta 0:00:00
ipywidgets already installed.
pydantic==1.10.13 already installed.


In [9]:
DATA_DIR = "/kaggle/input/agriculturedatasetnewmeta/MetaHack" # Default path for input files in Kaggle
OUTPUT_DIR = "/kaggle/working" # Default path for output in Kaggle
EMBEDDINGS_DIR = os.path.join(OUTPUT_DIR, "embeddings")
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 200

In [10]:
# Create output directories
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(EMBEDDINGS_DIR, exist_ok=True)

In [11]:
def extract_text_from_pdfs(pdf_directory: str) -> Dict[str, str]:
    """
    Extract text from all PDFs in the given directory
    """
    pdf_texts = {}
    
    for filename in os.listdir(pdf_directory):
        if filename.endswith('.pdf'):
            file_path = os.path.join(pdf_directory, filename)
            with open(file_path, 'rb') as file:
                pdf_reader = PyPDF2.PdfReader(file)
                text = ""
                for page in pdf_reader.pages:
                    text += page.extract_text() + "\n"
                pdf_texts[filename] = text
    
    return pdf_texts

In [12]:
def preprocess_text(text: str) -> str:
    """
    Clean and preprocess text from PDFs
    """
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    # Remove header/footer patterns (customize based on your PDFs)
    text = re.sub(r'Page \d+ of \d+', '', text)
    return text.strip()

In [13]:
def split_text_into_chunks(text: str, chunk_size: int = CHUNK_SIZE, chunk_overlap: int = CHUNK_OVERLAP) -> List[str]:
    """
    Split text into manageable chunks for embedding
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    return text_splitter.split_text(text)

In [14]:
def create_embeddings(chunks: List[str], source_documents: List[str]) -> FAISS:
    """
    Create embeddings for text chunks using HuggingFace embeddings and store in FAISS index
    """
    # Use a HuggingFace embedding model that works well with FAISS
    embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
    
    # Configure to use GPU if available
    model_kwargs = {'device': device}
    
    embeddings = HuggingFaceEmbeddings(
        model_name=embedding_model_name,
        model_kwargs=model_kwargs
    )
    
    # Create metadata for each chunk
    metadatas = [{"source": doc} for doc in source_documents]
    
    # Create and persist vector store
    vectorstore = FAISS.from_texts(
        texts=chunks,
        embedding=embeddings,
        metadatas=metadatas
    )
    
    # Save the FAISS index
    vectorstore.save_local(EMBEDDINGS_DIR)
    
    return vectorstore

In [37]:
def setup_retrieval_qa(vectorstore: FAISS) -> RetrievalQA:
    """
    Set up the Retrieval QA chain with conversation memory using HuggingFace model
    """
    # Create memory object
    memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True
    )
    
    # Load a language model that can run on GPU
    model_name = "google/flan-t5-base"  # Can be replaced with other models as needed
    
    # Initialize the model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    
    # Create pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.3,
        top_p=0.95,
        
    )
    
    # Create LLM
    llm = HuggingFacePipeline(pipeline=pipe)
    
    # Create retrieval QA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vectorstore.as_retriever(search_kwargs={"k": 5}),
        memory=memory,
        return_source_documents=True
    )
    
    return qa_chain

In [31]:
def process_query(query: str, qa_chain: RetrievalQA) -> Tuple[str, List[str]]:
    """
    Process user query and return response with source documents
    """
    # Add specific prompt for agricultural financial assistance
    enhanced_query = f"""
    Query: {query}
    
    If this question is about agricultural financial assistance or government support schemes, 
    please provide detailed information about eligibility criteria, application process, 
    deadlines, and contact information if available. Focus on practical steps farmers can take 
    to access this support.
    """
    
    # Get response from QA chain
    result = qa_chain(enhanced_query)
    
    answer = result["result"]
    source_docs = []
    
    # Extract source document references
    if "source_documents" in result:
        for doc in result["source_documents"]:
            if hasattr(doc, "metadata") and "source" in doc.metadata:
                source = doc.metadata["source"]
                if source not in source_docs:
                    source_docs.append(source)
    
    return answer, source_docs

In [32]:
def build_knowledge_base(pdf_directory):
    """
    Build the knowledge base from PDFs
    """
    pdf_texts = extract_text_from_pdfs(pdf_directory)
    all_chunks = []
    source_documents = []
    
    for filename, text in pdf_texts.items():
        processed_text = preprocess_text(text)
        chunks = split_text_into_chunks(processed_text)
        all_chunks.extend(chunks)
        source_documents.extend([filename] * len(chunks))
    
    print(f"Created {len(all_chunks)} text chunks from {len(pdf_texts)} documents")
    
    # Create vector store
    vectorstore = create_embeddings(all_chunks, source_documents)
    
    return vectorstore

In [33]:
def create_kaggle_ui():
    """
    Create a simple UI using IPython widgets for Kaggle notebooks
    """
    # Create output widget to display chat history
    chat_output = widgets.Output()
    
    # Create input widget for user queries
    query_input = widgets.Text(
        value='',
        placeholder='Ask about agricultural policies and financial support...',
        description='Query:',
        disabled=False,
        layout=widgets.Layout(width='80%')
    )
    
    # Create button to submit query
    submit_button = widgets.Button(
        description='Ask',
        disabled=False,
        button_style='primary',
        tooltip='Submit your query',
        icon='check'
    )
    
    # Create button to reset chat
    reset_button = widgets.Button(
        description='Reset Chat',
        disabled=False,
        button_style='danger',
        tooltip='Reset the chat history',
        icon='refresh'
    )
    
    # Create file upload widget for custom PDFs
    upload_widget = widgets.FileUpload(
        accept='.pdf',
        multiple=True,
        description='Upload PDFs',
        layout=widgets.Layout(width='80%')
    )
    
    # Display header
    display(HTML("<h1>Agricultural Financial Assistance Chatbot</h1>"))
    display(HTML("<h3>Ask questions about agricultural policies and government support schemes</h3>"))
    
    # Display widgets
    input_row = widgets.HBox([query_input, submit_button, reset_button])
    display(input_row)
    display(upload_widget)
    display(chat_output)
    
    # Store chat history
    chat_history = []
    
    # Function to handle query submission
    def handle_submit(button):
        query = query_input.value
        if not query:
            return
        
        # Clear input field
        query_input.value = ''
        
        with chat_output:
            # Display user query
            display(HTML(f"<div style='background-color:#f0f0f0; padding:10px; margin:5px; border-radius:5px;'><b>You:</b> {query}</div>"))
            
            # Process query and display response
            display(HTML("<div style='padding:10px; margin:5px; border-radius:5px;'><b>Assistant:</b> <i>Searching through agricultural policies...</i></div>"))
            
            try:
                answer, sources = process_query(query, qa_chain)
                
                # Clear the "Searching..." message
                clear_output(wait=True)
                
                # Display user query again (since we cleared output)
                display(HTML(f"<div style='background-color:#f0f0f0; padding:10px; margin:5px; border-radius:5px;'><b>You:</b> {query}</div>"))
                
                # Display the answer
                response_html = f"<div style='padding:10px; margin:5px; border-radius:5px;'><b>Assistant:</b> {answer}"
                
                if sources:
                    response_html += "<br><br><b>Sources:</b><ul>"
                    for source in sources:
                        response_html += f"<li>{source}</li>"
                    response_html += "</ul>"
                
                response_html += "</div>"
                display(HTML(response_html))
                
                # Add to chat history
                chat_history.append({"role": "user", "content": query})
                chat_history.append({"role": "assistant", "content": answer, "sources": sources})
                
            except Exception as e:
                # Handle errors
                clear_output(wait=True)
                display(HTML(f"<div style='background-color:#f0f0f0; padding:10px; margin:5px; border-radius:5px;'><b>You:</b> {query}</div>"))
                display(HTML(f"<div style='color:red; padding:10px; margin:5px; border-radius:5px;'><b>Error:</b> {str(e)}</div>"))
    
    # Function to handle chat reset
    def handle_reset(button):
        with chat_output:
            clear_output()
            chat_history.clear()
            display(HTML("<div style='padding:10px; margin:5px; border-radius:5px;'><i>Chat history has been reset.</i></div>"))
    
    # Function to handle file uploads
    def handle_upload_change(change):
        if not change.new:
            return
        
        with chat_output:
            for filename, file_info in change.new.items():
                # Save the uploaded file
                upload_path = os.path.join(OUTPUT_DIR, filename)
                with open(upload_path, 'wb') as f:
                    f.write(file_info['content'])
                
                display(HTML(f"<div style='padding:10px; margin:5px; border-radius:5px;'><i>Uploaded {filename}. Rebuilding knowledge base...</i></div>"))
            
            # Rebuild knowledge base with new PDFs
            try:
                global vectorstore, qa_chain
                vectorstore = build_knowledge_base(OUTPUT_DIR)
                qa_chain = setup_retrieval_qa(vectorstore)
                display(HTML("<div style='padding:10px; margin:5px; border-radius:5px;'><i>Knowledge base updated successfully!</i></div>"))
            except Exception as e:
                display(HTML(f"<div style='color:red; padding:10px; margin:5px; border-radius:5px;'><b>Error rebuilding knowledge base:</b> {str(e)}</div>"))
    
    # Register event handlers
    submit_button.on_click(handle_submit)
    reset_button.on_click(handle_reset)
    upload_widget.observe(handle_upload_change, names='value')

In [28]:
def main():
    global DATA_DIR, vectorstore, qa_chain  # Declare global variables
    
    print("Initializing Agricultural Policy RAG Chatbot...")
    
    # Check if the input directory exists, otherwise create it
    if not os.path.exists(DATA_DIR):
        print(f"Input directory not found at {DATA_DIR}")
        print(f"Creating directory at {OUTPUT_DIR}/temp_pdfs")
        os.makedirs(f"{OUTPUT_DIR}/temp_pdfs", exist_ok=True)
        DATA_DIR = f"{OUTPUT_DIR}/temp_pdfs"
        print("Please upload PDF files using the file upload widget")
    
    # Create some dummy files if needed for testing
    if len([f for f in os.listdir(DATA_DIR) if f.endswith('.pdf')]) == 0:
        print("No PDF files found. The system will initialize with an empty knowledge base.")
        print("Please upload PDF files using the file upload widget to build the knowledge base.")
        sample_text = "This is a sample agricultural policy document. It contains information about farm subsidies."
        with open(os.path.join(OUTPUT_DIR, "sample_policy.pdf"), "wb") as f:
            writer = PyPDF2.PdfWriter()
            writer.add_blank_page(width=595, height=842)  # A4 size
            writer.write(f)
    
    try:
        # First, see if we can load from saved embeddings
        if os.path.exists(EMBEDDINGS_DIR) and any(os.listdir(EMBEDDINGS_DIR)):
            print("Loading existing knowledge base...")
            embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
            model_kwargs = {'device': device}
            embeddings = HuggingFaceEmbeddings(
                model_name=embedding_model_name,
                model_kwargs=model_kwargs
            )
            vectorstore = FAISS.load_local(EMBEDDINGS_DIR, embeddings)
            print("Knowledge base loaded successfully!")
        else:
            print("Building knowledge base from PDFs...")
            vectorstore = build_knowledge_base(DATA_DIR)
            print("Knowledge base built successfully!")
    except Exception as e:
        print(f"Error loading/building knowledge base: {str(e)}")
        embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
        model_kwargs = {'device': device}
        embeddings = HuggingFaceEmbeddings(
            model_name=embedding_model_name,
            model_kwargs=model_kwargs
        )
        vectorstore = FAISS.from_texts(
            texts=["This is a placeholder document for agricultural policies."],
            embedding=embeddings,
            metadatas=[{"source": "placeholder.pdf"}]
        )
    
    qa_chain = setup_retrieval_qa(vectorstore)
    
    create_kaggle_ui()
    
    print("Agricultural Policy RAG Chatbot is ready!")


In [35]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


In [36]:
if __name__=="__main__":
    main()

Initializing Agricultural Policy RAG Chatbot...
Loading existing knowledge base...
Error loading/building knowledge base: The de-serialization relies loading a pickle file. Pickle files can be modified to deliver a malicious payload that results in execution of arbitrary code on your machine.You will need to set `allow_dangerous_deserialization` to `True` to enable deserialization. If you do this, make sure that you trust the source of the data. For example, if you are loading a file that you created, and no that no one else has modified the file, then this is safe to do. Do not set this to `True` if you are loading a file from an untrusted source (e.g., some random site on the internet.).


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

ValueError: The model has been loaded with `accelerate` and therefore cannot be moved to a specific device. Please discard the `device` argument when creating your pipeline object.